In [18]:
import numpy as np
import matplotlib.pyplot as plt

def simplified_bound(z, L):
    eps = 1e-3
    half_l = (L - 1) * (1 - eps) / 2
    offset = 0.5 if L % 2 == 0 else 0.0
    shift = np.tan(offset / half_l)
    z_hat = np.tanh(z + shift) * half_l - offset
    quantized = round_ste(z_hat)

    half_width = L // 2
    return quantized / half_width

def simplified_bound_no_shift(z, L):
    eps = 1e-3
    half_l = (L - 1) * (1 - eps) / 2
    offset = 0.5 if L % 2 == 0 else 0.0
    z_hat = np.tanh(z) * half_l - offset
    quantized = round_ste(z_hat)

    half_width = L // 2
    return quantized / half_width

def simplified_bound_no_shift_no_offset(z, L):
    eps = 1e-3
    half_l = (L - 1) * (1 - eps) / 2
    z_hat = np.tanh(z) * half_l

def simplified_bound_no_shift_no_offset_no_eps(z, L):
    half_l = (L - 1) / 2
    z_hat = np.tanh(z) * half_l

def simplified_bound_no_shift_no_offset_no_scale(z, L):
    z_hat = np.tanh(z)

def round_ste(z):
    z_hat = np.round(z)
    return z_hat

In [ ]:
z = np.linspace(-3, 3, 1000)

z_bound_even = simplified_bound(z, 4)
z_bound_even_no_shift = simplified_bound_no_shift(z, 4)
z_bound_even_no_shift_no_offset = simplified_bound_no_shift_no_offset(z, 4)
z_bound_even_no_shift_no_offset_no_scale = simplified_bound_no_shift_no_offset_no_scale(z, 4)

z_bound_even_no_shift_rounded = z_bound_even_no_shift
z_bound_even_rounded = z_bound_even

plt.figure(figsize=(10, 6))

# plt.plot(z, z_bound_even_no_shift_no_offset_no_scale, label='Step 1: Bound Function, L=4 (tanh w/ eps)', color='purple', linestyle='--')
# plt.plot(z, z_bound_even_no_shift_no_offset, label='Step 2: Bound Function, L=4 (tanh + scale)', color='red', linestyle='--')
plt.plot(z, z_bound_even_no_shift, label='Step 3: Bound Function, L=4 (tanh + scale + offset)', color='gold', linestyle='--')
plt.plot(z, z_bound_even, label='Step 4: Bound Function, L=4 (tanh + scale + offset + shift)', color='green')
plt.plot(z, z_bound_even_rounded, label='Step 5: Round', color='black')
plt.plot(z, z_bound_even_no_shift_rounded, label='Step 5: Round (No Shift)', color='grey', linestyle='--')

# Adding the title, labels, and legend
plt.title('Effect of Bound Function for Even L')
plt.xlabel('Input z')
plt.ylabel('Transformed z')
plt.legend()

# Adding grid for better readability
plt.grid(True)

# Display the combined graph
plt.show()

In [ ]:
z_bound_odd = simplified_bound(z, 5)

plt.figure(figsize=(10, 6))
plt.plot(z, z_bound_odd, label='Bound Function, L=5 (Odd)')
plt.title('Effect of Bound Function for Odd L')
plt.xlabel('Input z')
plt.ylabel('Transformed z')
plt.legend()
plt.grid(True)
plt.show()